<a href="https://colab.research.google.com/github/rcarrata/deeplearning_tf_examples/blob/master/11_Galaxy_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Galaxies

## Classifying Galaxies Using Convolutional Neural Networks
Around the clock, telescopes affixed to orbital satellites and ground-based observatories are taking millions of pictures of millions upon millions of celestial bodies. These data, of stars, planets and galaxies provide an invaluable resource to astronomers.

However, there is a bottleneck: until the data is annotated, it’s incredibly difficult for scientists to put it to good use. Additionally, scientists are usually interested in subsets of the data, like galaxies with unique characteristics.

In this project, you will build a neural network to classify deep-space galaxies. You will be using image data curated by Galaxy Zoo, a crowd-sourced project devoted to annotating galaxies in support of scientific discovery.

You will identify “odd” properties of galaxies. The data falls into four classes:

* [1,0,0,0] - Galaxies with no identifying characteristics.
* [0,1,0,0] - Galaxies with rings.
* [0,0,1,0] - Galactic mergers.
* [0,0,0,1] - “Other,” Irregular celestial bodies.





## Steps 

1. Because the dataset comprises over one thousand images, you’ll use a custom function, load_galaxy_data() to load the compressed data files into the Codecademy learning environment as NumPy arrays. Take a look at the shape of the data.

  Use .shape to print the dimensions of the input_data and labels.

  What does the last dimension of the data indicate about the image data? What does the last dimension of the labels indicate about the labels?

  Use print(input_data.shape) and print(labels.shape).

  Because the last dimension of the data is 3, you know that the image data is RGB/in color. Because the last dimension of the labels is 4, and there are four classes, you know that the labels are one-hot vectors. For example, [1,0,0,0] → Normal galaxy.

2. Next, divide the data into training and validation data, using sklearn’s train_test_split() function.

* Set the test_size argument to be 0.20.
* Shuffle the data.
* Set the random_state to be 222.
* Set stratify=labels. This ensures that ratios of galaxies in your testing data will be the same as in the original dataset.

  Your code should look something like this:

  ```
  x_train, x_valid, y_train, y_valid = train_test_split(input_data, labels, test_size=0.20, stratify=labels, shuffle=True, random_state=222)
  ```

3. Now, it’s time to preprocess the input.

  Define an ImageDataGenerator, and configure it so that the object will normalize the pixels using the rescale parameter.

  You can do this with the following line of code:

  ```
  data_generator = ImageDataGenerator(rescale=1./255)
  ```

4. Next, create two NumpyArrayIterators using the .flow(x,y,batch_size=?) method. We recommend using a batch size of 5. Significantly larger batch sizes may cause memory issues on the Codecademy platform.

  Create a training data iterator by calling .flow() on your training data and labels.

  Create a validation data iterator by calling .flow() on your training data and labels.

  Your code should use the .flow() method like this:

  ```
  training_iterator = data_generator.flow(x_train, y_train,batch_size=5)
  validation_iterator = data_generator.flow(x_valid, y_valid, batch_size=5)
  ```

5. Next, build your model, starting with the input shape and output layer.

  Create a tf.keras.Sequential model named model.

  Add a tf.keras.Input layer. Refer back to the shape of the data. What should the input shape be?

  Add a tf.keras.layers.Dense layer as your output layer. Make sure that it outputs 4 features, for the four classes (“Normal”,”Ringed”,”Merger”,”Other”).

  Remember to use a softmax activation on this final layer.

  Your input shape should be (128,128,3). This is because the images are 128 pixels tall, 128 pixels wide, and have 3 channels: Red, Green, and Blue.

  Your solution should add the layers to a Sequential model:

  ```
  model = tf.keras.Sequential()
  model.add(tf.keras.Input(shape=(128, 128, 3)))
  model.add(tf.keras.layers.Dense(4,activation="softmax"))
  ```

6. Before you finish designing your architecture, compile your model with an optimizer, loss, and metrics.

  Use model.compile(optimizer=?,loss=?, metrics=[?,?]) to compile your model.

  * Use tf.keras.optimizers.Adam with a learning_rate of 0.001.

  * Because the labels are one-hot categories, use tf.keras.losses.CategoricalCrossentropy() as your loss.

  * Set [tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()] as your metrics.

  Your code for compiling the model should look like this:

  ```
  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()])
  ```

7. Now, let’s go back and finish fleshing out your architecture. An architecture that works well on this task is two convolutional layers, interspersed with max pooling layers, followed by two dense layers:

  * Conv2D: 8 filters, each 3x3 with strides of 2
  * MaxPooling2D: pool_size=(2, 2), strides=2
  * Conv2D: 8 filters, each 3x3 with strides of 2
  * MaxPooling2D: pool_size=(2, 2), strides=2
  * Flatten Layer
  * Hidden Dense Layer with 16 hidden units
  * Output Dense Layer

  Try coding up this architecture yourself, using:

  * tf.keras.layers.Conv2D
  * tf.keras.layers.MaxPooling2D
  * tf.keras.layers.Flatten()
  * tf.keras.layers.Dense()

  Don’t forget to use “relu” activations for Dense and Conv2D hidden layers!

  The full architecture of the model can be defined like this:

  ```
  model = tf.keras.Sequential()
  model.add(tf.keras.Input(shape=(128, 128, 3)))
  model.add(tf.keras.layers.Conv2D(8, 3, strides=2, activation="relu")) 
  model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Conv2D(8, 3, strides=2, activation="relu")) 
  model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2,2), strides=(2,2)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(16, activation="relu"))
  model.add(tf.keras.layers.Dense(4, activation="softmax"))
  ```

8. At this point, your model should have 7,164 parameters. Use model.summary() to confirm this.

9. Use model.fit(...) to train your model.

  * The first argument should be your training iterator.

  * Set steps_per_epoch to be the length of your training data, divided by your batch size.

  * Set epochs to be 8.

  * Set validation_data to be your validation iterator.

  * Set validation_steps to be the length of your validation data, divided by your batch size.

  ```
  model.fit(
        training_iterator,
        steps_per_epoch=len(x_train)/5,
        epochs=8,
        validation_data=validation_iterator,
        validation_steps=len(x_valid)/5)
  ```

10. Now you can run your code to train the model. Training may take a minute or two. After training for twelve epochs, your model’s accuracy should be around 0.60-0.70, and your AUC should fall into the 0.80-0.90 range!

  What do these results mean?

  Your accuracy tells you that your model assigns the highest probability to the correct class more than 60% of the time. For a classification task with over four classes, this is no small feat: a random baseline model would achieve only ~25% accuracy on the dataset. Your AUC tells you that for a random galaxy, there is more than an 80% chance your model would assign a higher probability to a true class than to a false one.

11. You have successfully trained a Convolutional Neural Network to classify galaxies.

  Think you can do even better? If you would like, try tweaking your architecture. Can you find a better set of hyperparameters? Make sure to watch your parameter count: it’s easy to accidentally create a model with more than tens of thousands of parameters, which could overfit to your relatively small dataset (or crash the Learning Environment).

  Note that scores will fluctuate a bit, depending on how the weights are randomly initialized.

  Here are a few parameters that you could consider tweaking:

  * learning rate
  * number of convolutional layers
  * number of filters, strides, and padding type per layer
  * stride and pool_size of max pooling layers
  * size of hidden linear layers


12. BONUS Want to visualize how your convolutional neural network processes images?

  Take a look at visualize.py. It contains a function, visualize_activations().

  This function loads in sample data, uses your model to make predictions, and then saves the feature maps from each convolutional layer. These feature maps showcase the activations of each filter as they are convolved across the input.

  Try importing this function by adding the following to the end of train.py:

  ```
  from visualize import visualize_activations
  visualize_activations(model,YOUR_VALIDATION_ITERATOR)
  ```

  And then replace YOUR_VALIDATION_ITERATOR with the NumpyIterator you defined in task four.

  You are not required to understand the visualization code. However, the hint provides a summary.


  visualize_results takes your Keras model and the validation iterator and does the following:

  * It loads in a sample batch of data using your validation iterator.

  * It uses model.predict() to generate predictions for the first sample images.

  * Next, it compares those predictions with the true labels and prints the result.

  * It then saves the image and the feature maps for each convolutional layer using matplotlib.

In [ ]:
# train.py

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from visualize import visualize_activations

from sklearn.model_selection import train_test_split
from utils import load_galaxy_data

import app


input_data, labels = load_galaxy_data()

print(input_data.shape)
print(labels.shape)

x_train, x_valid, y_train, y_valid = train_test_split(
    input_data,
    labels,
    test_size = 0.20,
    shuffle=True,
    random_state=222,
    stratify=labels)

validation_data_generator = ImageDataGenerator(
  rescale = 1./255
)

training_iterator = validation_data_generator.flow(x_train, y_train, batch_size=5)

validation_iterator = validation_data_generator.flow(x_valid,y_valid,batch_size=5)

model = tf.keras.Sequential()

# Input Layer
model.add(tf.keras.Input(shape=(128,128,3)))

model.add(tf.keras.layers.Conv2D(8, 3, strides=2, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Conv2D(8, 3, strides=2, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16, activation="relu"))

# Output Layer
model.add(tf.keras.layers.Dense(4,activation="softmax"))

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()]
)

model.fit(
  training_iterator,
  steps_per_epoch=len(x_train)/5,
  epochs=8,
  validation_data=validation_iterator,
  validation_steps=len(x_valid)/5
)

visualize_activations(model,validation_iterator)


In [ ]:
# visualize.py

import tensorflow as tf
from matplotlib import pyplot as plt

#Visualizes convolutional layer activations
def visualize_activations(model, validation_iterator):

  #A keras model that will output our previous model's activations for each convolutional layer:
  activation_extractor = tf.keras.Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers if "conv2d" in layer.name])

  #Take matplotlib frame and remove axes.
  def clean_plot(plot):
    plot.axes.get_xaxis().set_visible(False)
    plot.axes.get_yaxis().set_visible(False)

  #Dict mapping from class numbers to string labels:
  class_names = {0:"Regular",1:"Ringed",2:"Merger",3:"Other"}

  #Loads a sample batch of data
  sample_batch_input,sample_labels = validation_iterator.next()
 
  #Grabs the first five images
  sample_batch_input = sample_batch_input[:5]
  sample_labels = sample_labels[:5]

  #Makes predictions using model.predict(x)
  sample_predictions = model.predict(sample_batch_input)

  #Iterate of images, predictions, and true labels
  for i,(image, prediction, label) in enumerate(zip(sample_batch_input, sample_predictions, sample_labels)):

    image_name = "Galaxy_{}".format(i)

    #Gets predicted class with highest probability

    predicted_class = tf.argmax(prediction).numpy()

    #Gets correct label
    actual_class = tf.argmax(label).numpy()

    print(image_name)
    print("\tModel prediction: {}".format(prediction))
    print("\tTrue label: {} ({})".format(class_names[actual_class], actual_class))
    print("\tCorrect:", predicted_class == actual_class)

    #Saves image file using matplotlib
    sample_image = image
    clean_plot(plt.imshow(sample_image))

    plt.title(image_name+" Predicted: {}, Actual: {}".format(class_names[predicted_class], class_names[actual_class]))
    plt.savefig('static/images/'+image_name+".png")
    model_layer_output = activation_extractor(tf.expand_dims(sample_image,0))
    
    plt.clf()

    #Iterates over each layer output
    for l_num,output_data in enumerate(model_layer_output):

      #Creates a subplot for each filter
      fig, axs = plt.subplots(1, output_data.shape[-1])
      
      #For each filter
      for i in range(output_data.shape[-1]):

        #Plots the filter's activations
        
        clean_plot(axs[i].imshow(output_data[0][:, :, i], cmap="gray"))
      plt.suptitle(image_name+" Conv {}".format(l_num),y=0.6)
      plt.savefig('static/images/' + image_name+ "Conv{}.png".format(l_num))
      plt.clf()

In [ ]:
# utils.py

import requests
import io
import numpy as np
import os

#Loads data from url
def make_request(url):
    print("Requesting data from {}...".format(url))
    response = requests.get('https://content.codecademy.com/courses/deeplearning-with-tensorflow/'+url)
    response.raise_for_status()
    response_data = io.BytesIO(response.content)
    return response_data
    
#Loads galaxy data
def load_galaxy_data():
  
  #If cached file not found, loads data from url
  if not os.path.isfile('./cached_data.npz'):
     response_data = make_request(url='galaxydata.npz')

     with open("cached_data.npz","wb") as save_file:
      save_file.write(response_data.read())
 
  #Load data using NumPy
  data = np.load('cached_data.npz')

  print("Successfully loaded galaxy data!")
  
  return data["data"],data["labels"]

In [ ]:
# app.py

import numpy as np
import matplotlib.pyplot as plt
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

#some routing for displaying the home page
@app.route('/')
@app.route('/home')
def plot_graph():
  return render_template('plt_tmpl.html', name = "Visualizations Created in Task 12", 
                         url1 ='static/images/Galaxy_0Conv0.png', 
                         url2='static/images/Galaxy_0Conv1.png', 
                         url3='static/images/Galaxy_0.png', 
                         url4 ='static/images/Galaxy_1Conv0.png', 
                         url5='static/images/Galaxy_1Conv1.png', 
                         rl6='static/images/Galaxy_1.png',
                         url7 ='static/images/Galaxy_2Conv0.png', 
                         url8='static/images/Galaxy_2Conv1.png', 
                         url9='static/images/Galaxy_2.png', 
                         url10 ='static/images/Galaxy_3Conv0.png', 
                         url11='static/images/Galaxy_3Conv1.png', 
                         url12='static/images/Galaxy_3.png', 
                         url13 ='static/images/Galaxy_4Conv0.png', 
                         url14='static/images/Galaxy_4Conv1.png', 
                         url15='static/images/Galaxy_4.png')